# Loop data and feature extraction

It's time. I have been a bit stuck on where to go next but I really think it's time to focus on feature extraction/feature engineering. So what features am I interested in? 

Well I think that a good place to start is to think about time scales. What are our time scales of interest? I'd like to be able to see daily trends. I'd also like to see daily segments of time - say, morning, afternoon, evening, night. And I want to be able to see all of the treatment settings to see how those may trigger various blood sugar responses.

## Features to extract
This is just an initial list based on things I can think of off the top of my head that might be nice to be able to visualize.

### Dependent variables
- SVG readings every 5 mins (basically the raw blood glucose)
- daily time in range, time above, and time below
- number of lows needing correction per day
- All the same stats as daily stats, but split over daily segments (am, pm, evening, night)

### Independent variables
- total daily insulin
- daily bolus amount
- daily basal amount
- carb ratios
- insulin correction factor
- basal rates
- basal adjustments (high/low)
- automatic boluses

## Approach and next steps
I've already extracted a few of these - either partially or fully. But it would be really good to have these various statistics collected into summary dataframes where I could run analysis on them. 

So next steps are to both extract features from the dataset and also organize those data for easier processing.

# Dependent variables
These are mostly something related to the blood glucose level, or a statistic derived from that. We already have a function for extracting daily time in and out of range so let's start there.

In [6]:
import sys
sys.path.append("../")

from mdb_tools import load_data as ld
from mdb_tools import schemas
from mdb_tools import loop_stats as oop
from mdb_tools import sugar_plots as sp

import pandas as pd
import numpy as np

from datetime import datetime
import pytz

import matplotlib.pyplot as plt
%matplotlib widget

import hvplot.pandas

In [2]:
# Things that may change (ie eventual function arguments)
yml_secrets_file = '../../secrets/mdb_secrets.yml'
time_zone = 'US/Eastern'

# Access the database using the yml secrets file, and get a specific set of "collections"
col_entries, col_treatments, col_profile, col_devicestatus = ld.get_collections(yml_secrets_file)

# Grab schemas
entries_schema, treatments_schema, devicestatus_schema = schemas.mdb_schemas()

# Load
df_entries0 = col_entries.find_pandas_all({}, schema=entries_schema)
df_treatments = col_treatments.find_pandas_all({}, schema=treatments_schema)
df_devicestatus = col_devicestatus.find_pandas_all({}, schema=devicestatus_schema)

# Remove duplicate entries from cgm date - keep only loop for now.
df_entries = df_entries0[df_entries0["device"]=="loop://Dexcom/G6/21.0"].copy()

# Convert entries time strings to datetime format 
df_entries["time"] = pd.to_datetime(df_entries["dateString"]).dt.tz_convert(time_zone)
df_entries.set_index("time", inplace = True, drop=False)
df_entries.sort_index(inplace=True)

# Convert device status time strings to datetime format
df_devicestatus["time"] = pd.to_datetime(df_devicestatus["created_at"]).dt.tz_convert(time_zone)

In [3]:
cur = col_treatments.find({'eventType':'Correction Bolus'})
t_list = []
for c in cur:
    t_list.append(c)

In [5]:
df_cgm_daily = oop.daily_cgm_stats(df_entries['time'], df_entries['sgv'], min_target=70, max_target=180)
df_cgm_daily.tail()


,yearday,time,pct_above,pct_below,pct_inrange
yearday,,,,,
2023-361,2023-361,2023-12-27,32.386364,2.272727,65.340909
2023-362,2023-362,2023-12-28,25.102881,3.703704,71.193416
2023-363,2023-363,2023-12-29,40.000000,0.000000,60.000000
2023-364,2023-364,2023-12-30,14.285714,4.761905,80.952381
2023-365,2023-365,2023-12-31,62.962963,0.000000,37.037037


In [7]:
# Add a time column that has time zone defined, make that the index
df_treatments["time"] = pd.to_datetime(df_treatments["timestamp"]).dt.tz_convert(time_zone)
df_treatments.set_index("time", inplace = True, drop=False)
df_treatments.sort_index(inplace=True)

## Computing Total Daily Insulin

### User defines/requests a time period and increment
The user would need to provide a start and end date/time (in pandas datetime format). They would also need to provide a step size/increment, which would be the time step at which analysis would occur. For example, User might select start and end dates (e.g., 1 Sep 2023 - 31 October 2023, or maybe just "last 2 months"). Then they might choose 24h as the step size. Other step sizes that could be analyzed would be hours or daily "segments" (morning, afternoon, evening, night). 

Loop through each of the increments within the requested time period. Within each of the subsets, compute and store the following information.

### Sum basal over requested time periods

### Add/subtract based on treatments
Within each of the looped segments, extract info from df_treatments data frame.

#### Temp basal
Multiply the hourly rate by the number of hours (duration) to get total delivered temp basal. 

#### Correction Bolus
Add the "insulin" value directly (it's not a rate so is not multiplied by duration).

#### Carb Correction
Use the carb ratio lookup (get_settings_at_time)

# Step 1: User selects date/time range

In [97]:
date_start = pd.Timestamp(year=2023, month=11, day=5)
date_end = pd.Timestamp(year=2023, month=12, day=30)

dates = pd.date_range(start=date_start, end=date_end, freq="H").tz_localize('US/Eastern', ambiguous='NaT')

# Custom function to remove the NaTs from dates (replace with nearest)
def replace_na_with_nearest_value(index):
    index = index.to_series()
    index = index.fillna(method='ffill')
    index = index.fillna(method='bfill')
    return index

# Replace "NaT" values with the nearest real value (backward fill)
filled_dates = replace_na_with_nearest_value(dates)


# Step 2: Sum the basal insulin over the time period


In [28]:
# Get basal rate at every time
hourly_basal_rate = oop.get_setting_at_times(filled_dates, col_profile, req_setting="basal")

# Create a new dataframe just for basal rates
df_basal = pd.DataFrame(index=dates)
df_basal['hourly_basal'] = hourly_basal_rate
df_basal['time'] = df_basal.index

# Group by date and sum within groups to get total basal insulin
daily_basal = df_basal.groupby([df_basal['time'].dt.date])['hourly_basal'].sum()


# Step 3: Sum "treatment" insulin

In [99]:
event_types = ["Carb Correction", "Correction Bolus", "Temp Basal", "Temporary Override"]
e_type = "Temp Basal"

for e in event_types:
    df_sub = df_treatments[df_treatments["eventType"] == e]
    match e:
        case "Correction Bolus": 
            df_sub = df_sub_date[df_sub_date["eventType"] == e].copy()
            insulin_total = df_sub.groupby([df_sub['time'].dt.date])['insulin'].sum()
        case "Temp Basal": 
            df_sub = df_sub_date[df_sub_date["eventType"] == e].copy()
            basal = oop.get_setting_at_times(df_sub['time'], col_profile, req_setting="basal")
            df_sub['basal_difference'] = np.multiply(np.subtract(df_sub['absolute'], basal), df_sub['duration']/60)
            basal_adjustment = df_sub.groupby([df_sub['time'].dt.date])['basal_difference'].sum()



In [23]:
# df_sub_date = df_treatments.loc[date_start:date_end].copy()

# # ######################################################################################################
# # #  Sum all the correction boluses for each day within the subset of requested days
# e = "Correction Bolus"
# df_sub = df_sub_date[df_sub_date["eventType"] == e]
# insulin_total = df_sub.groupby([df_sub['time'].dt.date])['insulin'].sum()

# # ######################################################################################################
# # #  Compute the total insulin difference due to temp basal adjustments, again grouped by day
# e = "Temp Basal"
# df_sub = df_sub_date[df_sub_date["eventType"] == e].copy()
# basal = oop.get_setting_at_times(df_sub['time'], col_profile, req_setting="basal")
# df_sub['basal_difference'] = np.multiply(np.subtract(df_sub['absolute'], basal), df_sub['duration']/60)
# basal_adjustment = df_sub.groupby([df_sub['time'].dt.date])['basal_difference'].sum()

